In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb


In [ ]:
suite.visual_summary_table()

In [168]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Modifier: adj', 'des…

In [2]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


In [3]:
editor = checklist.editor.Editor()
editor.tg

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [201]:
# [x for x in qs if 'better' in x[0]]

In [6]:
import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [7]:
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]

In [8]:
suite = TestSuite()

## Vocabulary

In [12]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
print(', '.join(professions))

journalist, historian, nurse, engineer, accountant, attorney, architect, artist, editor, actor, actress, analyst, intern, economist, entrepreneur, author, assistant, interpreter, investor, executive, organizer, escort, educator, agent, academic, activist, advisor, administrator, investigator, auditor


In [13]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
# professions

In [14]:
print(', '.join(editor.suggest('{first_name} {last_name} is a good {mask}.')[:30]))

guy, player, writer, actor, example, man, person, friend, kid, hitter, read, shooter, one, coach, dude, pick, teacher, shot, fighter, artist, student, quarterback, poet, character, reader, sport, name, bet, cook, reporter


In [15]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [16]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:mask} {noun}?', noun=nouns)[:50]))

effective, good, excellent, actual, ethical, active, outstanding, accredited, elite, ideal, honest, American, paid, acceptable, experienced, average, unusual, international, ordinary, Australian, real, amateur, professional, independent, accomplished, official, legitimate, better, OK, exceptional, accurate, bad, attractive, amazing, established, adequate, aspiring, important, okay, appropriate, authentic, unreliable, authorized, trusted, career, accidental, financial, former, sound, successful


In [17]:
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']

In [18]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0, name='Modifier: adj', capability='Vocabulary', 
          description = 'Adding an adjective makes questions non-duplicates')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Is person {a1, a2}?

In [19]:
print(', '.join(editor.suggest('Is John Wayne {mask}?')))
print()
print(', '.join(editor.suggest('Is John Wayne {a:mask}?')[:50]))

dead, gay, alive, right, Dead, real, Jewish, mad, wrong, correct, insane, back, crazy, related, OK, Right, immortal, evil, straight, interested, famous, racist, sane, next, President, alone, lying, relevant, okay, DEAD, bisexual, God, angry, guilty, ok, Alone, Wrong, ready, gone, innocent, King, human, Muslim, Black, assassinated, Batman, here, Real, dying, involved, homosexual, special, Famous, happy, free, murdered, Superman, president, joking, White, Back, Gay, Catholic, cool, safe, resurrected, killed, American, missing, corrupt, cursed, alright, single, Crazy, Mad, Satan, Missing, legit, shot, Evil, finished, worthy, doomed, sincere, aware, autistic, responsible, dangerous, Christian, Cool, Immortal, radioactive, remembered, Next, serious, cheating, different, popular, Legend, listening, now, illegitimate, homophobic, kidding, sick, smart, connected, trustworthy, Innocent, legal

racist, atheist, outlaw, asshole, actor, ape, American, inspiration, idiot, orphan, Christian, ancesto

In [20]:
adjs_without_overlap = ['dead', 'gay', 'Jewish', 'Christian', 'American', 'mad', 'immortal', 'evil', 'famous', 'racist', 'Muslim', 'white', 'black', 'English', 'autistic', 'Australian', 'trustworthy', 'an atheist', 'an anarchist', 'an inventor', 'Indian', 'Armenian', 'an astronaut', 'an immigrant']

In [21]:
t = editor.template((
    'Is {first_name} {last_name} {adj1}?',
    'Is {first_name} {last_name} {adj2}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='different adjectives', capability = 'Vocabulary',
          description='Same first and last name, different adjectives')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

person1 and person2 are different by first and last name

In [22]:
t = editor.template((
    'Is {first_name1} {last_name1} {adj}?',
    'Is {first_name2} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people', capability = 'Vocabulary',
          description='Different first and last name, same adjectives')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

person1 and person2 are different by first name only

In [23]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name2} {last_name} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v2', capability = 'Vocabulary',
          description='Different first name, same adjective and last name')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

person1 and person2 are different by last name only

In [24]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v3', capability = 'Vocabulary',
          description='Different last name, same adjective and first name')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

different animals

In [25]:
print(', '.join(editor.suggest('I have a pet {mask}.')))

cat, dog, rabbit, turtle, too, spider, now, rat, name, goat, lizard, dragon, named, pig, wolf, monkey, girl, squirrel, owl, also, here, called, there, tiger, python, friend, lab, phone, mix, bear, bird, …, boy, farm, duck, snake, zoo, owner, problem, fish, carrier, Shiva, shop, myself, companion, boyfriend, kitten, store, dinosaur, animal, watch, ring, project, deer, Labrador, Shepherd, tree, killer, lobster, mom, dish, X, bunny, girlfriend, shark, already, gun, car, puppy, one, computer, elephant, fetish, bug, brother, lover, tattoo, monitor, snail, cow, XD, Goat, mouse, somewhere, robot, sometimes, mother, syndrome, haha, pair, knife, life, carriage, Persian, cats, collar, that, diary, door, family, boo, number, show, assistant, toy, horse, chicken, laptop, server, tracker


In [59]:
animals = ['cat', 'dog', 'rabbit', 'turtle', 'spider', 'rat', 'goat', 'lizard', 'pig', 'monkey', 'squirrel', 'owl', 'snake', 'fish', 'lobster', 'snail', 'chicken']

In [60]:
print(', '.join(editor.suggest('Can I feed my {an} {mask}?', an=animals)))

food, pellets, eggs, water, again, worms, properly, now, meat, today, too, anything, outside, here, something, tonight, poop, fish, this, milk, directly, that, myself, rice, instead, nuts, there, live, steak, more, dinner, formula, treats, poison, regularly, correctly, rabbit, right, blood, enough, back, chicken, cheese, once, larvae, peanuts, feed, alive, honey, grass, feces, soup, anyway, scraps, bugs, what, alone, breakfast, oil, raw, well, chocolate, urine, corn, cereal, sugar, seeds, home, liver, free, better, protein, online, daily, cookies, carrots, lunch, yogurt, candy, cat, grain, seed, spinach, anymore, salad, fat, antibiotics, greens, Rice, it, indoors, …, shit, inside, money, toys, crap, organic, naturally, hay, bacon, some, cats, bird, bamboo, egg, breast, separately, rats, juice, vegetables, biscuits, waste, butter, crow, insulin, soy, sometimes, meal, mix, bones, yet, fly, appropriately, balls, privately, bananas, tuna, salmon, chickens, Milo, birds, bread, saliva, chick

In [89]:
editor.tg.antonyms('How can I make my cat more valuable?', 'valuable')

[]

In [61]:
food = ['eggs', 'water',  'worms', 'meat', 'poop', 'milk', 'rice', 'nuts', 'steak', 'formula',  'soup', 'bugs', 'oil', 'chocolate', 'corn', 'cereal', 'sugar', 'seeds', 'liver', 'cookies', 'carrots', 'yogurt', 'salad', 'greens', 'rice', 'bananas', 'tuna', 'apples', 'salmon', 'butter', 'insulin', 'soy']

In [40]:
t = editor.template((
    'Can I feed my {animal1} {food}?',
    'Can I feed my {animal2} {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Different animals' 
desc = 'Ask the same question about two different pet animals, expect prediction to be 0'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
test.run(new_pp)
test.summary(n=3)
suite.add(test)

Predicting 943 examples
Test cases:      943
Fails (rate):    69 (7.3%)

Example fails:
0.9 ('Can I feed my pig sugar?', 'Can I feed my goat sugar?')
----
0.9 ('Can I feed my dog sugar?', 'Can I feed my chicken sugar?')
----
0.8 ('Can I feed my lobster rice?', 'Can I feed my spider rice?')
----


In [120]:
idx = np.random.choice(np.where(labels == 1)[0], 5)
for i in idx:
    print(qs[i])

('Can you offer me any advice on how to lose weight?', 'How can I lose an extreme amount of weight?')
('What has been your best sexual experience?', "What was the best sexual experience you've ever had?")
('Was Dr Vinay Kumaran ragged in First year of MBBS?', 'Was Dr. Vinay Kumaran ragged in the first year of MBBS?')
('How the way to speak english fluently?', 'How could an English learner speak fluently?')
('How can I apply online for issuance of PAN card in India?', 'How do I apply and receive pan card in 48 hours with new facilities?')


In [76]:
[x for x,y in zip(qs, labels) if ' pet ' in x[0] and ' pet ' not in x[1] and y==1]

[('What are the best recommendations to live with a pet in an apartment?',
  'What are the best pets to have if you live in an apartment?'),
 ('What are the pros and cons of having your pet neutered?',
  'What are the pros and cons of neutering your dog?')]

In [79]:
t = editor.template((
    'My pet {animal1} eats {food}. Is it normal for non-human animals to eat {food}?',
    'My pet {animal2} eats {food}. Is it normal for non-human animals to eat {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Different animals' 
desc = 'Ask the same question about two different pet animals, expect prediction to be 0'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 943 examples
Test cases:      943
Fails (rate):    929 (98.5%)

Example fails:
0.0 ('My pet rat eats corn. Is it normal for non-human animals to eat corn?', 'My pet snail eats corn. Is it normal for non-human animals to eat corn?')
----
0.1 ('My pet chicken eats cereal. Is it normal for non-human animals to eat cereal?', 'My pet dog eats cereal. Is it normal for non-human animals to eat cereal?')
----
0.0 ('My pet owl eats sugar. Is it normal for non-human animals to eat sugar?', 'My pet lobster eats sugar. Is it normal for non-human animals to eat sugar?')
----


In [39]:
editor.visual_suggest('I broke my {mask} playing tennis.')

TemplateEditor(bert_suggests=['arm', 'leg', 'wrist', 'back', 'hand', 'neck', 'foot', 'ankle', 'nose', 'thumb',…

In [55]:
nouns = editor.selected_suggestions
nouns = list(set([x.rstrip('s') for x in nouns]))
', '.join(nouns)

'finger, forearm, feet, head, foot, elbow, nose, back, heart, shoulder, skull, toe, hip, neck, body, thumb, rib, knee, jaw, heel, thigh, ankle, arm, wrist, eye, spine, leg, butt, teeth, skin, bone, hand, face'

In [58]:
t = editor.template((
    'I hurt my {n} last time I played {sport}. {followup}',
    'I hurt my {n2} last time I played {sport}. {followup}',
    ),
    n=nouns,
    sport=['tennis', 'golf', 'soccer', 'football'],
    followup=['Is this going to impact my performance?',
              'Should I never play again?',
              'Is this a common injury?',
             'Is it normal to hurt this part of the body?',],
    remove_duplicates=True, 
    nsamples=1000)
name = 'Different injuries' 
desc = 'Question antecedent mentions differnet injuries, should not be duplicates.'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 965 examples
Test cases:      965
Fails (rate):    270 (28.0%)

Example fails:
0.9 ('I hurt my foot last time I played soccer. Should I never play again?', 'I hurt my knee last time I played soccer. Should I never play again?')
----
0.6 ('I hurt my back last time I played soccer. Is this going to impact my performance?', 'I hurt my elbow last time I played soccer. Is this going to impact my performance?')
----
0.9 ('I hurt my knee last time I played tennis. Is this a common injury?', 'I hurt my foot last time I played tennis. Is this a common injury?')
----


### Taxonomy

Antonyms

In [123]:
e

[(['modern'], ' How can I become more modern?', 2.6216812133789062),
 (['advanced'], ' How can I become more advanced?', 4.115582466125488)]

In [158]:
tmp = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.synonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        tmp.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(',\n'.join([str(tuple(x)) for x in tmp]))

('spiritual', 'religious'),
('anxious', 'nervous'),
('activist', 'militant'),
('lonely', 'alone', 'solitary'),
('strict', 'rigid', 'stern'),
('important', 'authoritative', 'significant'),
('progressive', 'liberal', 'imperfect'),
('organized', 'organised', 'direct'),
('religious', 'spiritual'),
('positive', 'confident'),
('corrupt', 'corrupted'),
('critical', 'decisive', 'vital'),
('tolerant', 'resistant', 'liberal', 'kind'),
('thoughtful', 'attentive'),
('healthy', 'intelligent', 'sound', 'respectable'),
('rude', 'primitive', 'crude'),
('kind', 'tolerant'),
('nervous', 'anxious'),
('consistent', 'uniform', 'logical', 'coherent'),
('hungry', 'thirsty'),
('aware', 'mindful'),
('evil', 'vicious'),
('mean', 'average', 'hateful'),
('needy', 'impoverished'),
('charitable', 'sympathetic', 'benevolent'),
('vocal', 'outspoken'),
('disconnected', 'confused', 'fragmented'),
('demanding', 'exact'),
('individual', 'single', 'private', 'person'),
('independent', 'autonomous'),
('grateful', 'thankful

In [208]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'), ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'), ('humble', 'modest'), ('courageous', 'brave'), ]

In [180]:

t = editor.template([
    (
    'How can I become more {x[0]}?',
    'How can I become more {x[1]}?',
    ),
    (
    'How can I become more {x[1]}?',
    'How can I become more {x[0]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become {a:x[0]} person?',
    'How can I become {a:x[1]} person?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become {a:x[0]} person?',
    ),
],
    unroll=True,
    x=synonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'synonyms' 
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 6000 examples
Test cases:      6000
Fails (rate):    1186 (19.8%)

Example fails:
0.3 ('How can I become more spiritual?', 'How can I become more religious?')
----
0.0 ('How can I become less vocal?', 'How can I become less outspoken?')
----
0.0 ('How can I become more outspoken?', 'How can I become more vocal?')
----


In [264]:
import re
def replace_pairs(pairs):
    def replace_z(text):
        ret = []
        for x, y in pairs:
            t = re.sub(r'\b%s\b' % x, y, text )
            if t != text:
                ret.append((text, t))
            if y == 'smart':
                continue
            t = re.sub(r'\b%s\b' % y, x, text )
            if t != text:
                ret.append((text, t))
        return list(set(ret))
    return replace_z

In [265]:
t = Perturb.perturb(list(all_questions), replace_pairs(synonyms), nsamples=200, keep_original=False)
test = INV(t.data, threshold=0.1)
test.run(new_pp)
test.summary(n=3)


Predicting 168 examples
Test cases:      168
Fails (rate):    0 (0.0%)


Antonyms

In [95]:
opps = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.tg.antonyms('How can I become more %s?' % a, a)
    if e:
#         print(a, e[0][0][0])
        opps.append((a, e[0][0][0]))

In [183]:
opps = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.antonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        opps.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(','.join([str(tuple(x)) for x in opps]))

('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative', 'neutral'),('defensive', 'offensive'),('rude', 'civil', 'polite'),('hungry', 'thirsty'),('optimistic', 'pessimistic'),('stupid', 'intelligent', 'smart'),('negative', 'positive'),('individual', 'common'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'lean', 'thin'),('bad', 'good'),('cautious', 'brave'),('organic', 'functional'),('smart', 'stupid'),('hopeful', 'hopeless'),('conservative', 'liberal', 'progressive'),('emotional', 'intellectual'),('specific', 'general'),('conspicuous', 'invisible'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('shy', 'confident'),('irresponsible', 'responsible'),('courageous', 'fearful'),('capitalist', 'socialist')


In [184]:
opps = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

In [173]:

t = editor.template([(
    'How can I become more {x[0]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = 'More != Less' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Taxonomy',
          description=desc)
test.run(new_pp)
test.summary(n=3)
suite.add(test)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    790 (39.5%)

Example fails:
0.9 ('How can I become less secular?', 'How can I become more secular?')
----
0.7 ('How can I become more irresponsible?', 'How can I become less irresponsible?')
----
0.6 ('How can I become more pessimistic?', 'How can I become less pessimistic?')
----


Exception: There is already a test named More != Less suite. Run with overwrite=True to overwrite

In [174]:

t = editor.template([(
    
    'How can I become more {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = 'More X = Less antonym(X)' 
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1349 (67.5%)

Example fails:
0.0 ('How can I become more active?', 'How can I become less passive?')
----
0.2 ('How can I become more religious?', 'How can I become less secular?')
----
0.0 ('How can I become less powerless?', 'How can I become more powerful?')
----


DIR version (kinda bad, won't add to suite)

In [280]:
t = Perturb.perturb(list(all_questions), replace_pairs(opps), nsamples=500, keep_original=False)
test = DIR(t.data, expect=Expect.eq(0), agg_fn='all')
test.run(new_pp)
test.summary(n=5)

Predicting 519 examples
Test cases:      500
Fails (rate):    193 (38.6%)

Example fails:
0.6 ('Is it a bad time to buy a condo or a house in the Bay Area in 2017?', 'Is it a good time to buy a condo or a house in the Bay Area in 2017?')
----
0.9 ('I am a 26 year old male and I love hanging out with people who are older than me. Is that a bad sign?', 'I am a 26 year old male and I love hanging out with people who are older than me. Is that a good sign?')
----
0.9 ('Do people regret their tattoos often? Even if its a good tattoo?', 'Do people regret their tattoos often? Even if its a bad tattoo?')
----
0.7 ('Were Jews responsible for the killing of Jesus?', 'Were Jews irresponsible for the killing of Jesus?')
----
0.6 ('Which is a good inpatient drug and alcohol rehab center near Warren County GA?', 'Which is a bad inpatient drug and alcohol rehab center near Warren County GA?')
----


## Robustness

In [29]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
def wrap_apply_to_both(fn, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn

Typos

In [42]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.add_typos), nsamples=500)
test = INV(t.data)
# test.run(new_pp)
# test.summary(3)
suite.add(test, 'add one typo', 'Robustness', 'TODO_DESCRIPTION')

Contractions

In [44]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.contractions, both=True), nsamples=500)
test = INV(**t)
test.run(new_pp)
test.summary(3)
suite.add(test, 'contractions', 'Robustness', 'TODO_DESCRIPTION')

Predicting 1479 examples
Test cases:      500
Fails (rate):    13 (2.6%)

Example fails:
0.6 ('Can any body explain about meshing in FEA?', "Altair Hypermesh: What is meshing in ansys? Please answer in detail in lay man's language?")
0.4 ('Can any body explain about meshing in FEA?', "Altair Hypermesh: What's meshing in ansys? Please answer in detail in lay man's language?")

----
0.5 ('How do you know if someone is following you back on Snapchat?', "How do you check if someone is following you on Snapchat if you don't follow them?")
0.1 ('How do you know if someone is following you back on Snapchat?', 'How do you check if someone is following you on Snapchat if you do not follow them?')

----
0.7 ("Who was the kindest/friendliest celebrity you've ever met?", "Who's the nicest celebrity you've ever met?")
0.4 ('Who was the kindest/friendliest celebrity you have ever met?', 'Who is the nicest celebrity you have ever met?')
0.5 ("Who was the kindest/friendliest celebrity you've ever met?

## Ner

### Change same name, number, location in both

Names

In [143]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both

def change_each_wrapper(fn):
    def change_one(qs, **kwargs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True, **kwargs)
        c2 = fn(q2, seed=seed, meta=True, **kwargs)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        ret = []
        ret.extend([(q1_, str(q2)) for q1_, m1_ in zip(c1, m1) if m1_[0] in str(q2)])
        ret.extend([(str(q1), q2_) for q2_, m2_ in zip(c2, m2) if m2_[0] in str(q1)])
        return ret
    return change_one
    
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(**t)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change same name in both questions', 'NER', 'TODO_DESCRIPTION')

Locs

In [47]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_location), nsamples=500)
test = INV(**t)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change same location in both questions', 'NER', 'TODO_DESCRIPTION')

Predicting 5092 examples
Test cases:      500
Fails (rate):    52 (10.4%)

Example fails:
0.9 ('Is it safe for non-Muslims foreigners to visit Pakistan?', 'How safe is travelling in Pakistan?')
0.2 ('Is it safe for non-Muslims foreigners to visit Japan?', 'How safe is travelling in Japan?')
0.3 ('Is it safe for non-Muslims foreigners to visit Australia?', 'How safe is travelling in Australia?')

----
0.0 ('What does Pakistani people think about India?', 'What do pakisthani people think about India?')
1.0 ('What does Pakistani people think about Netherlands?', 'What do pakisthani people think about Netherlands?')
0.9 ('What does Pakistani people think about Gibraltar?', 'What do pakisthani people think about Gibraltar?')

----
1.0 ('Which are the best thrilling Hollywood movies?', 'What are the best Hollywood movies?')
0.2 ('Which are the best thrilling Orland Park movies?', 'What are the best Orland Park movies?')
0.4 ('Which are the best thrilling Lakeland movies?', 'What are the best

Numbers

In [53]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_number), nsamples=500)
test = INV(**t)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change same number in both questions', 'NER', 'TODO_DESCRIPTION')

Predicting 4958 examples
Test cases:      500
Fails (rate):    29 (5.8%)

Example fails:
0.7 ('What changes can we expect in education systems in the next 50 years?', 'How will teaching and the classroom change in the next 30-50 years?')
0.1 ('What changes can we expect in education systems in the next 54 years?', 'How will teaching and the classroom change in the next 30-54 years?')

----
0.2 ('How do I get started for cds exam 2016?', 'How I can apply for cds exam 2016?')
0.9 ('How do I get started for cds exam 1900?', 'How I can apply for cds exam 1900?')
0.7 ('How do I get started for cds exam 1862?', 'How I can apply for cds exam 1862?')

----
0.4 ('Which is better iPhone 6s+ or iPhone 7?', 'Which is the best phone iPhone 6s plus or 7?')
0.8 ('Which is better iPhone 6s+ or iPhone 6?', 'Which is the best phone iPhone 6s plus or 6?')
0.8 ('Which is better iPhone 6s+ or iPhone 6?', 'Which is the best phone iPhone 6s plus or 6?')

----


### Change name, loc, number in only one where orig prediction is duplicate

Changing only first names

In [144]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
test = DIR(**t, expect=expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 9948 examples
Test cases:      500
After filtering: 289 (57.8%)
Fails (rate):    256 (88.6%)

Example fails:
1.0 ('Is Star Wars or Harry Potter a bigger franchise?', 'What is a bigger franchise universe: Harry Potter or Star Wars?')
1.0 ('Is Star Wars or Harry Potter a bigger franchise?', 'What is a bigger franchise universe: Jeremiah Potter or Star Wars?')
1.0 ('Is Star Wars or Harry Potter a bigger franchise?', 'What is a bigger franchise universe: Aaron Potter or Star Wars?')

----
1.0 ('What should I do if Donald Trump wins the election?', 'What will you do now that Donald Trump is the President?')
1.0 ('What should I do if Donald Trump wins the election?', 'What will you do now that Matthew Trump is the President?')
1.0 ('What should I do if Donald Trump wins the election?', 'What will you do now that Bryan Trump is the President?')

----
1.0 ('What will happen if Donald Trump wins?', 'Why is Donald Trump supported so much? What will happen if he wins the election?')
1.

Changing first and last names

In [59]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=1500)
test = DIR(**t, expect=expect_fn)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change first and last name in only one question', 'NER', 'TODO_DESCRIPTION')


Predicting 23540 examples
Test cases:      1500
After filtering: 598 (39.9%)
Fails (rate):    279 (46.7%)

Example fails:
1.0 ('Donald Trump says he saw thousands cheering in New Jersey when the Twin Towers came down on 9/11. Is this claim likely to be accurate?', 'Is Donald Trump lying about seeing thousands and thousands of Muslims cheering in New Jersey when the World Trade Center came down on 9/11?')
0.8 ('David Ross says he saw thousands cheering in New Jersey when the Twin Towers came down on 9/11. Is this claim likely to be accurate?', 'Is Donald Trump lying about seeing thousands and thousands of Muslims cheering in New Jersey when the World Trade Center came down on 9/11?')
0.7 ('Daniel Price says he saw thousands cheering in New Jersey when the Twin Towers came down on 9/11. Is this claim likely to be accurate?', 'Is Donald Trump lying about seeing thousands and thousands of Muslims cheering in New Jersey when the World Trade Center came down on 9/11?')

----
1.0 ('"Is it pos

Locs

In [61]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_location), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_location), nsamples=1500)
test = DIR(**t, expect=expect_fn)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change loc in only one question', 'NER', 'TODO_DESCRIPTION')

Predicting 23080 examples
Test cases:      1500
After filtering: 491 (32.7%)
Fails (rate):    219 (44.6%)

Example fails:
1.0 ('Is CAT after b.tech in mechanical engineering useful?', 'Will I have a great career if I go for B.Tech in mechanical engineering in a state level college in India?')
1.0 ('Is CAT after b.tech in mechanical engineering useful?', 'Will I have a great career if I go for B.Tech in mechanical engineering in a state level college in United States?')
1.0 ('Is CAT after b.tech in mechanical engineering useful?', 'Will I have a great career if I go for B.Tech in mechanical engineering in a state level college in Georgia?')

----
1.0 ('Can the Supreme Court of India interfere on the banned of 500 and 1000 notes by the government?', 'Can supreme court in India cancel government decision on demonetization?')
1.0 ('Can the Supreme Court of India interfere on the banned of 500 and 1000 notes by the government?', 'Can supreme court in St. Martin (French part) cancel governme

numbers

In [145]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_number), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_number), nsamples=1500)
test = DIR(**t, expect=expect_fn)
test.run(new_pp)
test.summary(3)
suite.add(test, 'change number in only one question', 'NER', 'TODO_DESCRIPTION')

Predicting 9519 examples
Test cases:      500
After filtering: 307 (61.4%)
Fails (rate):    261 (85.0%)

Example fails:
1.0 ('Who is more likely to be the President of India in 2017?', 'Who should be elected as president of India in 2017?')
1.0 ('Who is more likely to be the President of India in 2017?', 'Who should be elected as president of India in 2046?')
1.0 ('Who is more likely to be the President of India in 2187?', 'Who should be elected as president of India in 2017?')

----
1.0 ('Which will be the best buy between Yamaha r3 and ninja 300?', 'Which one is better, yamaha r3 or kawasaki ninja 300?')
1.0 ('Which will be the best buy between Yamaha r3 and ninja 300?', 'Which one is better, yamaha r3 or kawasaki ninja 301?')
1.0 ('Which will be the best buy between Yamaha r3 and ninja 301?', 'Which one is better, yamaha r3 or kawasaki ninja 300?')

----
1.0 ('What will be the repercussions of banning Rs 500 and Rs 1000 notes on Indian economy?', 'How will the ban of 1000 and 500 ru

Exception: There is already a test named change number in only one question suite. Run with overwrite=True to overwrite

In [63]:
### Change 

### Keep entities, fill in with BERT gibberish

In [67]:
def mask_gibberish(question):
    ents = question.ents
    if not ents:
        return None
    wp = [x.text for x in question if x.tag_ in ['WP', 'WRB', 'WDT']]
    if not wp:
        wp = question[0].text
    else:
        wp = wp[0]
    ents = [x.text for x in ents]
    ents[-1] = ents[-1] + '?'
    template = ' {mask} '.join([wp] + ents)
    gibberish = editor.template(template).data[:5]
#     return gibberish
    ret = [(question.text, x) for x in gibberish if question.text != x ]
    return ret
def gibberish_both(qs):
    q1, q2 = qs
    ret = []
    x1 = mask_gibberish(q1)
    if x1:
        ret.extend(x1)
    x2 = mask_gibberish(q2)
    if x2:
        ret.extend(x2)
    return ret
#     ret =  [(question.text, x[1]) for x in tg.fill_in_between([wp] + ents) if question.text != x[1]]
#     ret = [x for x in ret if len(x[1].split()) < len(x[0].split()) -4][:5]
#     return ret 

In [68]:
t = Perturb.perturb(parsed_qs, gibberish_both, nsamples=500)
expect_false = Expect.eq(0)
test = DIR(**t, expect=expect_false)
test.run(new_pp)
test.summary(3)
suite.add(test, 'Keep entities, fill in with BERT gibberish', 'NER', 'TODO_DESCRIPTION')

Predicting 4744 examples
Test cases:      500
Fails (rate):    143 (28.6%)

Example fails:
0.0 ('What is the cost of 1 rupee note bundle from 1993?', 'What is the value of a 1994 1 Rupee note?')
1.0 ('What is the value of a 1994 1 Rupee note?', 'What the 1994 Indian Rupee?')
0.9 ('What is the value of a 1994 1 Rupee note?', 'What Was 1994 The Rupee?')

----
0.0 ('Is this years surge in home runs in MLB due to performance enhancing drugs?', 'I am male, 17 years, 6′ 2″ and 84 kg. Is it okay? How can I stop my height growth?')
1.0 ('I am male, 17 years, 6′ 2″ and 84 kg. Is it okay? How can I stop my height growth?', 'How can 17 years old 6′ 2″ weigh 84 kg?')
1.0 ('I am male, 17 years, 6′ 2″ and 84 kg. Is it okay? How can I stop my height growth?', 'How … 17 years … 6′ 2″ � 84 kg?')

----
0.0 ('What is the best tag line for Quora?', 'What is best tag line in history?')
1.0 ('What is the best tag line for Quora?', 'What on Quora?')
0.8 ('What is the best tag line for Quora?', 'What was Quor

In [69]:
test.summary(3)

Test cases:      500
Fails (rate):    143 (28.6%)

Example fails:
1.0 ('Is there a positive side to the future president Donald Trump?', 'What positive change would a Donald Trump presidency bring?')
0.9 ('Is there a positive side to the future president Donald Trump?', 'Is he Donald Trump?')

----
1.0 ('Is ISI better than RAW?', 'What makes ISI better than RAW?')
0.9 ('What makes ISI better than RAW?', 'What distinguishes ISI from RAW?')

----
1.0 ('What should be my resolution for 2017?', "What are some of your New Year's resolutions for 2017?")
1.0 ("What are some of your New Year's resolutions for 2017?", "What about New Year's Day 2017?")
1.0 ("What are some of your New Year's resolutions for 2017?", "What About New Year's Day 2017?")

----


## Temporal

Is != used to be

In [70]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Did {first_name} {last_name} use to be {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'is != used to be', 'Temporal', 'TODO_DESCRIPTION')

Predicting 999 examples
Test cases:      999
Fails (rate):    613 (61.4%)

Example fails:
0.9 ('Is Amanda Bennett an executive?', 'Did Amanda Bennett use to be an executive?')
----
0.7 ('Is Alexis Barnes an analyst?', 'Did Alexis Barnes use to be an analyst?')
----
0.9 ('Is Emma Jackson an architect?', 'Did Emma Jackson use to be an architect?')
----


Is != becoming

In [71]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} becoming {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'is != becoming', 'Temporal', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    181 (18.1%)

Example fails:
0.8 ('Is Amber Green an employee?', 'Is Amber Green becoming an employee?')
----
0.5 ('Is Ashley Watson an advocate?', 'Is Ashley Watson becoming an advocate?')
----
0.6 ('Is Alexander Cook an activist?', 'Is Alexander Cook becoming an activist?')
----


Before != after

In [72]:
t = editor.template((
    'What was {first_name} {last_name}\'s life before becoming {a:noun}?',
    'What was {first_name} {last_name}\'s life after becoming {a:noun}?'
),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'before != after', 'Temporal', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    1000 (100.0%)

Example fails:
1.0 ("What was Taylor Morales's life before becoming an architect?", "What was Taylor Morales's life after becoming an architect?")
----
1.0 ("What was Tyler Mitchell's life before becoming a person?", "What was Tyler Mitchell's life after becoming a person?")
----
0.9 ("What was Jennifer Hall's life before becoming an assistant?", "What was Jennifer Hall's life after becoming an assistant?")
----


In [73]:
print(', '.join(editor.suggest('Is is {mask} to eat after 10pm?')))

it, normal, OK, there, safe, best, ok, okay, acceptable, reasonable, permissible, legal, proper, wrong, healthy, good, enough, better, important, appropriate, left, allowed, possible, dinner, vegetarian, what, recommended, going, food, illegal, supposed, smart, unhealthy, hard, popular, hot, necessary, vegan, typical, not, free, right, required, fun, time, meant, something, one, kosher, customary, fair, mandatory, available, common, cheaper, alright, easier, suitable, feasible, polite, difficult, cool, certain, sensible, affordable, advisable, fine, fashionable, like, nice, unsafe, public, wise, decent, sufficient, harder, advised, traditional, usual, healthier, anyone, permitted, eating, easy, life, trendy, someone, forbidden, lawful, expensive, safer, new, everyone, compulsory, dangerous, worse, needed, realistic, temptation, essential, adequate, impossible, hungry, rude, risky, considered, prudent, taboo, different, tempting


In [193]:
mid = ['normal', 'ok', 'safe', 'dangerous', 'acceptable', 'reasonable', 'proper', 'wrong', 'healthy', 'important']

In [75]:
print(', '.join(editor.suggest('Is is {mid} to {mask} after 10pm?', mid=mid)))

sleep, drink, eat, work, leave, drive, go, smoke, stay, call, vote, talk, read, retire, visit, write, celebrate, post, do, party, watch, study, exercise, travel, fish, start, stop, walk, play, strike, disappear, be, tweet, live, continue, shop, rise, speak, relax, die, finish, blog, run, move, act, wait, text, cook, return, dance, bed, fly, quit, report, swim, pray, ask, rest, answer, close, gamble, enter, crash, think, marry, come, kill, look, meet, feed, operate, propose, vanish, dress, pee, linger, change, check, remain, dinner, protest, nap, withdraw, wake, fight, begin, publish, arrive, attend, skate, disturb, end, gather, complain, chat, exist, worry, know, cry, shower, cancel, see, open, perform, queue, respond, resign, exit, book, commute, function, avoid, film, buy, awake, binge, snack, clean, vape, park, happen, comment, blackout, occur, cycle, have, paint, vomit, panic, refuse, sit, say, resume, demonstrate, disconnect, kiss, feel, lose, search, hunt, bar, breathe, take, sur

In [76]:
activity = ['drink', 'sleep', 'drive', 'work', 'eat', 'smoke', 'walk', 'read', 'party', 'talk', 'exercise', 'celebrate', 'text', 'tweet', 'run', 'dance', 'swim', 'cook', 'pray', 'pee', 'rest']

In [77]:
t = editor.template(('Is it {mid} to {activity} before {hour}{ampm}?','Is it {mid} to {activity} after {hour}{ampm}?'),
                activity=activity,
                mid=mid,
                hour=[str(x) for x in range(1, 12)],
                ampm=['am', 'pm'],
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'before != after, activity', 'Temporal', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    979 (97.9%)

Example fails:
1.0 ('Is it dangerous to smoke before 11am?', 'Is it dangerous to smoke after 11am?')
----
1.0 ('Is it reasonable to pee before 1am?', 'Is it reasonable to pee after 1am?')
----
1.0 ('Is it normal to rest before 2am?', 'Is it normal to rest after 2am?')
----


## Negation

In [194]:
mid2 = mid + ['legal', 'awkward', 'socially acceptable']

In [195]:
print(', '.join(editor.suggest('Is it {mid} to {mask} in {country}?', mid=mid2)[:100]))

live, work, vote, be, study, stay, marry, drive, travel, fight, eat, exist, speak, remain, pray, die, campaign, visit, protest, serve, meet, move, squat, write, hunt, shop, gather, demonstrate, smoke, swim, race, perform, fly, gamble, fish, pee, reside, play, worship, settle, sleep, party, march, report, train, teach, operate, surf, act, talk, drink, volunteer, hate, kill, murder, dance, arrive, migrate, cook, land, rape, experiment, gay, preach, exercise, practice, tan, farm, invest, strike, stand, survive, think, believe, paint, spy, start, barbecue, date, lie, propose, hide, rule, identify, appear, vacation, offend, beef, breed, burn, shoot, ask, compete, build, refugee, transgender, sue, queer, discriminate, Muslim


In [196]:
things = ['work', 'vote', 'travel', 'marry', 'drive', 'study', 'protest', 'campaign', 'fight', 'gamble', 'hunt', 'pray', 'smoke', 'fish', 'murder', 'invest', 'pee', 'march', 'worship', 'volunteer', 'surf', 'shoot', 'dance', 'camp', 'preach', 'spy', 'be gay', 'lie', 'divorce', 'discriminate']

In [197]:
t = editor.template(('Is it {mid} to {activity} in {country}?','Is it {mid} not to {activity} in {country}?'),
                activity=things,
                mid=mid2,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Simple negation: Is it normal to X / not to X in country', 'Negation', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    232 (23.2%)

Example fails:
0.9 ('Is it dangerous to divorce in Malta?', 'Is it dangerous not to divorce in Malta?')
----
0.6 ('Is it socially acceptable to murder in Gambia?', 'Is it socially acceptable not to murder in Gambia?')
----
0.8 ('Is it reasonable to pee in China?', 'Is it reasonable not to pee in China?')
----


In [82]:
t = editor.template((
    'What are things {a:noun} should worry about?',
    'What are things {a:noun} should not worry about?',
),
                noun=nouns,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Simple negation: things to worry / not to worry about', 'Negation', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)


In [179]:
t = editor.template([(
    'How can I become {a:x[0]} person?',
    'How can I become a person who is not {x[1]}?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become a person who is not {x[0]}?',
    ),
],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = '' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Negation',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    462 (23.1%)

Example fails:
0.6 ('How can I become an unhappy person?', 'How can I become a person who is not happy?')
----
0.6 ('How can I become a secure person?', 'How can I become a person who is not insecure?')
----
1.0 ('How can I become a rude person?', 'How can I become a person who is not polite?')
----


## Coref

In [84]:
t = editor.template(
    [(
        'If {male} and {female} were alone, do you think he would reject her?',
        'If {male} and {female} were alone, do you think she would reject him?',
    ),
        (
        'If {female} and {male} were alone, do you think he would reject her?',
        'If {female} and {male} were alone, do you think she would reject him?',
    )
    ],
    remove_duplicates=True, 
    nsamples=1000,
    unroll=True)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Simple coref: he and she', 'Coref', 'TODO_DESCRIPTION')

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1648 (82.4%)

Example fails:
0.9 ('If Alicia and Liam were alone, do you think he would reject her?', 'If Alicia and Liam were alone, do you think she would reject him?')
----
0.9 ('If Chelsea and Jared were alone, do you think he would reject her?', 'If Chelsea and Jared were alone, do you think she would reject him?')
----
0.9 ('If Brooke and Henry were alone, do you think he would reject her?', 'If Brooke and Henry were alone, do you think she would reject him?')
----


In [85]:
t = editor.template(
    [(
        'If {male} and {female} were married, would his family be happy?',
        'If {male} and {female} were married, would {female}\'s family be happy?',
    ),(
        'If {male} and {female} were married, would her family be happy?',
        'If {male} and {female} were married, would {male}\'s family be happy?',
    ),
    ]
        ,
    unroll=True,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Simple coref: his and her', 'Coref', 'TODO_DESCRIPTION')

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1977 (98.8%)

Example fails:
0.9 ('If Luke and Savannah were married, would her family be happy?', "If Luke and Savannah were married, would Luke's family be happy?")
----
0.9 ('If Richard and Isabella were married, would her family be happy?', "If Richard and Isabella were married, would Richard's family be happy?")
----
1.0 ('If Justin and Brianna were married, would his family be happy?', "If Justin and Brianna were married, would Brianna's family be happy?")
----


## SRL

In [86]:
print(', '.join(editor.suggest('Who is the best {mask} in the world?')))

chef, pizza, boxer, player, footballer, athlete, rapper, actor, singer, cook, magician, robot, coach, beer, cyclist, wrestler, drummer, musician, quarterback, hacker, dog, baker, fighter, journalist, restaurant, steak, teacher, doctor, gamer, band, shooter, DJ, football, person, horse, photographer, driver, fisherman, burger, coffee, lawyer, writer, food, dancer, student, wine, artist, man, surgeon, comedian, trainer, VPN, vegetarian, programmer, team, game, vegan, guitarist, goalkeeper, server, guy, cricket, sport, engineer, dentist, waiter, cheese, bartender, fish, chicken, goalie, car, manager, computer, referee, hunter, guitar, basketball, tennis, astronaut, mathematician, sandwich, blogger, accountant, TV, soldier, sniper, job, runner, barbecue, chocolate, scientist, friend, cat, gun, pitcher, judge, AI, philosopher, butcher, painter, golf, camel, translator, piano, worker, farmer, conductor, politician, bike


In [87]:
thing = ['chef', 'boxer', 'player', 'footballer', 'athlete', 'rapper', 'actor', 'singer', 'cook', 'magician', 'coach', 'cyclist', 'wrestler', 'drummer', 'musician', 'quarterback', 'hacker', 'baker', 'fighter', 'journalist', 'teacher', 'doctor', 'gamer', 'husband', 'DJ', 'person', 'man', 'woman', 'surgeon', 'comedian', 'trainer', 'programmer', 'guitarist', 'goalkeeper']

In [88]:
print(', '.join(editor.suggest('Who do {mask} think is the the best {thing} in the world?', thing=thing)))

you, YOU, people, we, they, I, readers, You, guys, u, your, fans, ya, others, experts, Americans, scientists, men, voters, some, the, students, analysts, authors, everyone, friends, conservatives, players, critics, Canadians, he, judges, folks, women, all, most, historians, i, coaches, supporters, members, journalists, researchers, celebrities, many, viewers, liberals, participants, audiences, Australians, respondents, editors, Republicans, artists, writers, U, pundits, yo, comedians, gamers, reporters, economists, teachers, parents, veterans, commentators, consumers, anyone, users, independents, kids, pros, millennials, each, individuals, YOUR, those, athletes, doctors, candidates, reviewers, ye, investigators, both, politicians, scholars, not, two, philosophers, atheists, agents, humans, stars, ladies, police, feminists, insiders, yours, any, archaeologists, leaders, Christians, seniors, Democrats, contestants, psychologists, adults, guests, teams, astronomers, competitors, professio

In [89]:
subjects = ['you', 'people', 'readers', 'guys', 'fans', 'experts', 'scientists', 'Americans', 'students', 'men', 'voters', 'authors', 'conservatives', 'women', 'Canadians', 'analysts', 'critics', 'judges', 'artists', 'researchers', 'liberals', 'historians', 'Australians', 'journalists', 'Republicans', 'coaches', 'parents', 'kids', 'economists', 'reporters', 'consumers', 'veterans', 'doctors']

In [90]:
print(', '.join(editor.suggest('Who do {subjects} think is the the {mask} {thing} in the world?', thing=thing, subjects=subjects)[:50]))

best, greatest, worst, smartest, top, biggest, fastest, strongest, toughest, finest, happiest, coolest, hottest, deadliest, richest, hardest, safest, better, oldest, brightest, leading, great, youngest, Greatest, largest, weakest, superior, favorite, easiest, elite, highest, most, busiest, quickest, tallest, wealthiest, favourite, perfect, premier, ultimate, next, smallest, first, right, only, dominant, BEST, heaviest, newest, good


In [91]:
best = ['best', 'greatest', 'worst', 'top', 'smartest', 'strongest', 'finests', 'happiest', 'coolest', 'richest', 'leading', 'brightest', 'premier', 'ultimate', 'dominant']

In [92]:
t = editor.template((
    'Who do {subjects} think is the {best} {thing} in the world?',
    'Who is the {best} {thing} in the world according to {subjects}?'
),
    subjects=subjects,
    best=best,
    thing=thing,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=1)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Who do X think - Who is the ... according to X', 'SRL', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    131 (13.1%)

Example fails:
0.2 ('Who do guys think is the best cyclist in the world?', 'Who is the best cyclist in the world according to guys?')
----
0.1 ('Who do Americans think is the dominant DJ in the world?', 'Who is the dominant DJ in the world according to Americans?')
----
0.4 ('Who do consumers think is the dominant fighter in the world?', 'Who is the dominant fighter in the world according to consumers?')
----


In [93]:
print(', '.join([str(x) for x in editor.suggest('Are {mask} smaller than {a}?', a=['bananas', 'dogs', 'cars', 'cats', 'elephants'])][:100]))

humans, cats, mice, dogs, pigs, people, you, birds, chickens, cows, sheep, rats, fish, bears, rabbits, monkeys, elephants, we, snakes, bees, bats, lions, children, puppies, spiders, babies, ants, insects, they, kittens, dolphins, butterflies, frogs, animals, robots, turtles, ducks, trees, bugs, kids, flies, worms, whales, mosquitoes, crabs, cars, plants, sharks, dinosaurs, beetles, horses, tigers, wolves, primates, cattle, chimpanzees, goats, apes, men, deer, reptiles, rodents, mammals, balls, pets, ponies, carrots, seals, potatoes, apples, mushrooms, dragons, ticks, boys, toys, girls, us, computers, things, stones, women, brains, eggs, bulls, calves, Indians, dwarves, bananas, houses, burgers, flowers, shells, beans, diamonds, coins, trolls, guys, drones, bacteria, Lions


In [94]:
things = editor.suggest('Are {mask} smaller than {a}?',a=['bananas', 'dogs', 'cars', 'cats', 'elephants'] )[:100]

In [99]:
print(', '.join([str(x) for x in editor.suggest('Are {a} {mask} than {a2}?', a=things)][:100]))

smarter, better, bigger, faster, stronger, cooler, different, smaller, worse, larger, tougher, more, taller, safer, slower, wiser, heavier, weaker, healthier, happier, lighter, nicer, less, older, greater, quicker, cheaper, harder, shorter, cleaner, longer, quieter, brighter, easier, darker, hotter, higher, closer, other, colder, thicker, younger, louder, warmer, lower, sharper, deeper, wider, lesser, softer, broader, simpler, thinner, intelligent, clearer, stranger, important, dangerous, farther, rather, fewer, richer, smoother, finer, differently, smart, tighter, poorer, earlier, sooner, superior, real, related, stupid, wealthier, anymore, inferior, normal, valuable, dumb, interesting, beautiful, weird, narrower, big, further, stricter, strange, human, safe, powerful, rare, MORE, fuller, dead, special, alive, similar, old, newer


In [100]:
comp = ['better', 'worse', 'cheaper', 'bigger', 'louder', 'longer', 'larger', 'smaller', 'warmer', 'colder', 'thicker', 'lighter', 'heavier']

Order doesn't matter for comparison

In [101]:
t = editor.template((
    'Are {t1} {comp} than {t2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'What is {comp}, {t2} or {t1}?'
),
    t = things,
    comp = comp,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=1)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Order does not matter for comparison', 'SRL', 'TODO_DESCRIPTION')

Predicting 990 examples
Test cases:      990
Fails (rate):    982 (99.2%)

Example fails:
0.0 ('Are men better than bulls?', 'What is better, bulls or men?')
----
0.0 ('Are kittens worse than babies?', 'What is worse, babies or kittens?')
----
0.0 ('Are eggs lighter than burgers?', 'What is lighter, burgers or eggs?')
----


Order matters for asymetric relations

In [102]:
# editor.template('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]

In [103]:
print(', '.join(editor.suggest('Is {first_name1} {mask} to {first_name2}?', remove_duplicates=True)[:100]))
print()
print(', '.join(editor.suggest('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]))

related, married, close, engaged, talking, lying, connected, speaking, closer, loyal, linked, attracted, going, referring, listening, important, faithful, similar, mean, responding, writing, hostile, true, dead, tied, next, bound, known, indebted, lied, proposing, closest, opposed, truthful, committed, right, up, turning, kind, Married, kin, proposed, attached, abusive, gay, good, real, returning, allergic, straight, available, relevant, talk, attractive, fair, written, nice, dangerous, crazy, new, alive, coming, on, friendly, devoted, cruel, entitled, special, superior, answer, addicted, reacting, out, back, born, supposed, equal, father, drawn, happy, happening, answering, acceptable, nicer, spoken, getting, open, decent, familiar, heir, guilty, talked, mentioned, gone, sympathetic, helpful, insane, down, pregnant, equivalent

or, really, and, with, actually, not, an, Really, still, like, /, &, born, a, from, called, real, indeed, now, the, dating, truly, after, even, married, to, ma

In [104]:
symmetric = ['dating', 'married to', 'close to', 'engaged to', 'connected to', 'married to', 'friends with', 'related to', 'an acquaintance of']

In [105]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = symmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=1)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Order does not matter for symmetric relations', 'SRL', 'TODO_DESCRIPTION')

Predicting 988 examples
Test cases:      988
Fails (rate):    756 (76.5%)

Example fails:
0.0 ('Is Jonathan engaged to Natalie?', 'Is Natalie engaged to Jonathan?')
----
0.1 ('Is Dylan married to Kevin?', 'Is Kevin married to Dylan?')
----
0.0 ('Is Jeremy related to Kelly?', 'Is Kelly related to Jeremy?')
----


In [106]:
asymmetric = ['hurting', 'lying to', 'loyal to', 'faithful to', 'proposing to', 'indebted to', 'abusive to', 'using', 'expecting', 'beating', 'punching', 'raising', 'poisoning', 'protecting', 'kidnapping']

In [107]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = asymmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'Order matters for asymmetric relations', 'SRL', 'TODO_DESCRIPTION')

Predicting 988 examples
Test cases:      988
Fails (rate):    243 (24.6%)

Example fails:
0.9 ('Is Aaron poisoning David?', 'Is David poisoning Aaron?')
----
0.8 ('Is Alyssa proposing to Hannah?', 'Is Hannah proposing to Alyssa?')
----
1.0 ('Is Charles using David?', 'Is David using Charles?')
----


More traditional SRL

In [108]:
print(', '.join(editor.suggest('Did John buy the {mask}?', remove_duplicates=True)[:100]))

farm, stake, house, property, company, land, rights, ticket, papers, newspaper, book, island, estate, ranch, boat, horse, paper, shares, tickets, business, gun, books, newspapers, yacht, team, ship, stock, idea, plot, contract, Bible, phone, franchise, shotgun, store, horses, cattle, church, place, game, factory, castle, club, painting, bank, tract, rifle, manuscript, car, campaign, Ark, slaves, plane, school, money, guns, sword, plantation, beer, trust, election, film, ring, building, station, twins, deal, loan, time, tractor, home, letter, IRA, plan, title, watch, story, letters, rest, cows, piece, diary, brewery, charter, diamond, mortgage, hospital, site, post, operation, insurance, railroad, cow, coins, Dell, case, children, wine, telephone, dog


In [109]:
obj = ['farm', 'house', 'property', 'company', 'land', 'ticket', 'newspaper', 'book', 'island', 'estate', 'ranch', 'boat', 'horse', 'paper', 'business', 'gun', 'game', 'factory', 'castle', 'painting', 'rifle', 'car', 'school', 'building']

In [110]:
print(', '.join(editor.suggest('Did John {mask} the {obj}?', obj=obj, remove_duplicates=True)[:100]))

buy, get, take, sell, have, lose, own, leave, see, keep, want, use, win, need, steal, manage, find, miss, receive, handle, inherit, purchase, abandon, seize, control, run, return, remember, finish, break, enjoy, know, hold, give, drop, claim, recover, move, touch, save, land, pull, on, destroy, share, start, rebuild, crash, call, stop, afford, hit, acquire, join, like, bought, deliver, lead, remove, clear, fix, sign, split, quit, watch, bring, play, survive, left, carry, sold, shoot, maintain, close, grab, settle, reclaim, accept, retain, enter, complete, change, possess, raise, make, wreck, beat, flip, catch, nail, build, forget, ruin, recall, try, retrieve, surrender, notice, mention, got


In [111]:
import pattern
import pattern.en
verbs = ['buy', 'purchase', 'sell', 'leave', 'own', 'take', 'keep', 'want', 'lose', 'destroy', 'inherit', 'find', 'use', 'need', 'receive', 'return', 'like', 'enjoy', 'abandon', 'manage', 'remember', 'miss', 'move', 'seize', 'steal']
a = pattern.en.tenses('stolen')[0]
verbs = [(v, pattern.en.conjugate(v, *a)) for v in verbs]
verbs[3] = ('leave', 'left')
verbs

[('buy', 'bought'),
 ('purchase', 'purchased'),
 ('sell', 'sold'),
 ('leave', 'left'),
 ('own', 'owned'),
 ('take', 'taken'),
 ('keep', 'kept'),
 ('want', 'wanted'),
 ('lose', 'lost'),
 ('destroy', 'destroyed'),
 ('inherit', 'inherited'),
 ('find', 'found'),
 ('use', 'used'),
 ('need', 'needed'),
 ('receive', 'received'),
 ('return', 'returned'),
 ('like', 'liked'),
 ('enjoy', 'enjoyed'),
 ('abandon', 'abandoned'),
 ('manage', 'managed'),
 ('remember', 'remembered'),
 ('miss', 'missed'),
 ('move', 'moved'),
 ('seize', 'seized'),
 ('steal', 'stolen')]

In [112]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was the {obj} {verb[1]} by {first_name}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=1)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'traditional SRL: active / passive swap', 'SRL', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    115 (11.5%)

Example fails:
0.2 ('Did Erin lose the newspaper?', 'Was the newspaper lost by Erin?')
----
0.0 ('Did Nicholas leave the horse?', 'Was the horse left by Nicholas?')
----
0.4 ('Did Sarah lose the school?', 'Was the school lost by Sarah?')
----


In [113]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was {first_name} {verb[1]} by the {obj}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'traditional SRL: wrong active / passive swap', 'SRL', 'TODO_DESCRIPTION')

Predicting 1000 examples
Test cases:      1000
Fails (rate):    962 (96.2%)

Example fails:
0.9 ('Did Adam lose the painting?', 'Was Adam lost by the painting?')
----
1.0 ('Did Mary miss the car?', 'Was Mary missed by the car?')
----
1.0 ('Did Nathan seize the paper?', 'Was Nathan seized by the paper?')
----


With people

In [114]:
print(', '.join(editor.suggest('Does {first_name} {mask} {first_name2}?', remove_duplicates=True)[:100]))

know, marry, kill, love, hate, remember, like, want, and, or, have, date, need, see, meet, tell, Know, find, recognize, choose, understand, forgive, trust, mention, blame, follow, murder, miss, bother, ask, deserve, save, beat, mean, attack, prefer, dislike, resemble, become, survive, support, get, help, visit, kiss, leave, accept, recognise, married, own, notice, be, hurt, call, replace, hit, take, fear, catch, reject, clone, divorce, believe, kidnap, Love, /, Remember, adopt, succeed, question, influence, killed, recall, shoot, contact, raise, respect, rape, use, judge, include, eat, v, owe, confront, join, KILL, despise, Like, sponsor, hire, crush, control, threaten, stalk, destroy, Want, &, defeat, lose


In [115]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name2} {verb[1]} by {first_name}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=1)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'traditional SRL: active / passive swap with people', 'SRL', 'TODO_DESCRIPTION')

Predicting 998 examples
Test cases:      998
Fails (rate):    681 (68.2%)

Example fails:
0.4 ('Does Rebecca notice Amy?', 'Is Amy noticed by Rebecca?')
----
0.0 ('Does Victoria attack Austin?', 'Is Austin attacked by Victoria?')
----
0.4 ('Does Joseph love Mary?', 'Is Mary loved by Joseph?')
----


In [116]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name} {verb[1]} by {first_name2}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(**t, labels=0)
test.run(new_pp)
test.summary(n=3)
suite.add(test, 'traditional SRL: wrong active / passive swap with people', 'SRL', 'TODO_DESCRIPTION')

Predicting 994 examples
Test cases:      994
Fails (rate):    974 (98.0%)

Example fails:
1.0 ('Does Scott bother Emily?', 'Is Scott bothered by Emily?')
----
1.0 ('Does Abigail deserve Eric?', 'Is Abigail deserved by Eric?')
----
0.9 ('Does Joshua understand Robert?', 'Is Joshua understood by Robert?')
----


In [118]:
suite.run(new_pp, overwrite=True)

Running Adding an adjective makes questions non-duplicates
Predicting 999 examples
Running non-overlapping adjectives, same first and last name
Predicting 957 examples
Running same adjectives, different first and last name
Predicting 983 examples
Running same adjectives, different first but same last name
Predicting 991 examples
Running same adjectives, same first but different last name
Predicting 986 examples
Running add one typo
Predicting 900 examples
Running contractions
Predicting 1479 examples
Running change same name in both questions
Predicting 5422 examples
Running change same location in both questions
Predicting 5092 examples
Running change same number in both questions
Predicting 4958 examples
Running change first name in only one question
Predicting 23540 examples
Running change first and last name in only one question
Predicting 23540 examples
Running change loc in only one question
Predicting 23080 examples
Running change number in only one question
Predicting 22900 exa

In [163]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Modifier: adj', 'des…

In [119]:
suite.summary(n=3)

Vocabulary

Adding an adjective makes questions non-duplicates
Test cases:      999
Fails (rate):    780 (78.1%)

Example fails:
1.0 ('Is Katherine Ward an executive?', 'Is Katherine Ward an excellent executive?')
----
0.9 ('Is Nicholas Kelly an engineer?', 'Is Nicholas Kelly an elite engineer?')
----
1.0 ('Is Sean King an entrepreneur?', 'Is Sean King a real entrepreneur?')
----


non-overlapping adjectives, same first and last name
Test cases:      957
Fails (rate):    3 (0.3%)

Example fails:
0.6 ('Is Jordan Cooper Indian?', 'Is Jordan Cooper autistic?')
----
0.5 ('Is Kyle Diaz Armenian?', 'Is Kyle Diaz an immigrant?')
----
0.9 ('Is Zachary Reyes American?', 'Is Zachary Reyes Indian?')
----


same adjectives, different first and last name
Test cases:      983
Fails (rate):    49 (5.0%)

Example fails:
0.8 ('Is Christopher Thomas dead?', 'Is Jeffrey Richardson dead?')
----
0.8 ('Is Rebecca Clark mad?', 'Is Emily Bennett mad?')
----
0.7 ('Is Kayla Sanders white?', 'Is Heather Clark wh

In [120]:
suite.save('/home/marcotcr/tmp/qqp_suite.pkl')

PicklingError: Can't pickle <class 'checklist.test_types.INV'>: it's not the same object as checklist.test_types.INV